<a href="https://colab.research.google.com/github/Juliana001/Utilizades-Computacionais/blob/main/pdf_colorido_para_pretoebranco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF
!pip install google-colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 70.0 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import fitz  # PyMuPDF
import cv2
import numpy as np
from google.colab import drive
import os

# Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

# Caminho para o PDF colorido no Google Drive
input_pdf_path = '/content/drive/MyDrive/preto&branco/Lista_problemas_02-cap02_2.21-a-2.42.pdf'
output_pdf_path = '/content/drive/MyDrive/preto&branco/seu_pdf_preto_sobre_branco.pdf'

# Abrir o arquivo PDF
pdf_document = fitz.open(input_pdf_path)

# Criar um novo PDF para salvar as páginas processadas
new_pdf = fitz.open()

# Fator de escala para melhorar a resolução da imagem
scale = 2  # Ajuste esse valor conforme necessário para obter a resolução desejada

# Loop através de todas as páginas do PDF
for page_num in range(pdf_document.page_count):
    # Obter a página
    page = pdf_document.load_page(page_num)

    # Converter a página para imagem com o fator de escala
    pix = page.get_pixmap(matrix=fitz.Matrix(scale, scale))

    # Converter a imagem para uma matriz de bytes (imagem bruta)
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)

    # Verificar se a imagem tem 4 canais (RGBA) e converter para RGB
    if img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

    # Converter a imagem para escala de cinza
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Aplicar a binarização (preto e branco), garantindo que o fundo seja branco e o texto seja preto
    _, black_text_img = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY)

    # Redimensionar a imagem para a resolução original da página
    black_text_img = cv2.resize(black_text_img, (pix.width, pix.height), interpolation=cv2.INTER_NEAREST)

    # Salvar a imagem em um arquivo temporário
    image_path = f"page_{page_num}.png"
    cv2.imwrite(image_path, black_text_img)

    # Criar uma nova página no PDF com o mesmo tamanho
    rect = page.rect  # Retorna o tamanho original da página
    new_page = new_pdf.new_page(width=rect.width, height=rect.height)  # Criar nova página com o mesmo tamanho

    # Inserir a imagem processada na nova página
    new_page.insert_image(rect, filename=image_path)

    # Remover a imagem temporária
    os.remove(image_path)

# Salvar o novo PDF no caminho especificado
new_pdf.save(output_pdf_path)
new_pdf.close()

# Fechar o PDF original
pdf_document.close()

print("Conversão completa. O PDF com fundo branco e conteúdo preto, mantendo a resolução, foi salvo no Google Drive.")


Mounted at /content/drive
Conversão completa. O PDF com fundo branco e conteúdo preto, mantendo a resolução, foi salvo no Google Drive.
